## install dependencies

In [3]:
!pip install pandas
!pip install pystan==2.19.1.1
!pip install prophet

 ## Import dependencies

In [1]:
import pandas as pd
# from prophet import Prophet
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib.pyplot'

## Load and prepare dataframes

In [14]:
df_btc = pd.read_csv("/opt/dev/trading/pfe/DRL-trading-bot/dataset/binance-ZILUSDT-1h.csv")
df = df_btc[['Date', 'Close']].copy()

df.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)

AttributeError: module 'pandas' has no attribute 'read_csv'

In [16]:
df['ds']

0        2017-08-17 04:00:00.000000000
1        2017-08-17 05:00:00.000000000
2        2017-08-17 06:00:00.000000000
3        2017-08-17 07:00:00.000000000
4        2017-08-17 08:00:00.000000000
                     ...              
38218              2021-12-31 20:00:00
38219              2021-12-31 21:00:00
38220              2021-12-31 22:00:00
38221              2021-12-31 23:00:00
38222              2022-01-01 00:00:00
Name: ds, Length: 38223, dtype: object

## fit model

In [4]:
MARGIN = 3822
WINDOW_SIZE = 10

In [5]:
df.loc[:MARGIN]

,ds,y
0,2017-08-17 04:00:00.000000000,4308.83
1,2017-08-17 05:00:00.000000000,4315.32
2,2017-08-17 06:00:00.000000000,4324.35
3,2017-08-17 07:00:00.000000000,4349.99
4,2017-08-17 08:00:00.000000000,4360.69
...,...,...
3818,2018-01-23 14:00:00.000000000,10459.55
3819,2018-01-23 15:00:00.000000000,10750.00
3820,2018-01-23 16:00:00.000000000,11334.96
3821,2018-01-23 17:00:00.000000000,11129.62


In [25]:
model = Prophet()

# initial fit
model.fit(df.loc[:MARGIN])

# Make prediction and fitting model with correct result
predict_steps = 1
initial_step = MARGIN

for i in range(predict_steps):
    future_df = model.make_future_dataframe(periods=WINDOW_SIZE)

    print('predicted future frame ============')
    display(future_df)

    print('future frame ============')
    # display(df.loc[i+MARGIN:i+MARGIN+WINDOW_SIZE])

    # model.fit(df.loc[i+MARGIN])


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


predicted future frame ============


,ds
0,2017-08-17 04:00:00
1,2017-08-17 05:00:00
2,2017-08-17 06:00:00
3,2017-08-17 07:00:00
4,2017-08-17 08:00:00
...,...
3828,2018-01-29 18:00:00
3829,2018-01-30 18:00:00
3830,2018-01-31 18:00:00
3831,2018-02-01 18:00:00


Initial log joint probability = -46.7026
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       9606.04     0.0677539       2549.03           1           1      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       9898.46     0.0149572       1985.18      0.4649      0.4649      223   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        9984.4      0.092372       1941.83           1           1      343   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399         10043     0.0495599       1785.56           1           1      459   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       10072.5     0.0021052       316.601      0.9951      0.9951      577   
    Iter      log prob        ||dx||      ||grad||       alpha  

## test predictions

In [6]:

train_df = df.loc[:MARGIN]

model = Prophet()

# initial fit
model.fit(train_df)


future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)

display(train_df)
display(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


Initial log joint probability = -46.7026
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       9606.04     0.0677539       2549.03           1           1      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       9898.46     0.0149572       1985.18      0.4649      0.4649      223   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        9984.4      0.092372       1941.83           1           1      343   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399         10043     0.0495599       1785.56           1           1      459   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       10072.5     0.0021052       316.601      0.9951      0.9951      577   
    Iter      log prob        ||dx||      ||grad||       alpha  

,ds,y
0,2017-08-17 04:00:00.000000000,4308.83
1,2017-08-17 05:00:00.000000000,4315.32
2,2017-08-17 06:00:00.000000000,4324.35
3,2017-08-17 07:00:00.000000000,4349.99
4,2017-08-17 08:00:00.000000000,4360.69
...,...,...
3818,2018-01-23 14:00:00.000000000,10459.55
3819,2018-01-23 15:00:00.000000000,10750.00
3820,2018-01-23 16:00:00.000000000,11334.96
3821,2018-01-23 17:00:00.000000000,11129.62


,ds,yhat,yhat_lower,yhat_upper
0,2017-08-17 04:00:00,4030.029631,3118.020800,4962.859533
1,2017-08-17 05:00:00,4037.794892,3018.715032,5002.740129
2,2017-08-17 06:00:00,4047.012482,3039.167455,4994.268884
3,2017-08-17 07:00:00,4048.496424,3055.875110,5016.084385
4,2017-08-17 08:00:00,4040.285677,3070.222160,4955.090696
...,...,...,...,...
3819,2018-01-23 15:00:00,11924.647876,10920.414214,12868.424496
3820,2018-01-23 16:00:00,11916.462161,10895.170731,12905.131766
3821,2018-01-23 17:00:00,11915.813132,10975.231179,12933.353310
3822,2018-01-23 18:00:00,11920.487575,10996.720622,12882.170673


In [14]:
forecast.columns

Index(['ds', 'trend', 'yhat_lower', 'yhat_upper', 'trend_lower', 'trend_upper',
       'additive_terms', 'additive_terms_lower', 'additive_terms_upper',
       'daily', 'daily_lower', 'daily_upper', 'weekly', 'weekly_lower',
       'weekly_upper', 'multiplicative_terms', 'multiplicative_terms_lower',
       'multiplicative_terms_upper', 'yhat'],
      dtype='object')

In [ ]:
def plot_tests(train, forcast):
    fig = plt.figure()
    times = forcast[]
    plt.plot()